# Convert the CF data into the SWDE format
1. This is done per dataset type (train, develop, test)
2. Make sure to select only the positive subset (pages containin at least one PAST CLIENT annotation text)
3. Select the Tags (default = PAST_CLIENT)
---
After that, this notebook will:
1. Clean a bit the data
 - By adjusting the name of domains by removing any hyphen
 - Making sure the name of the domain doesnt contain parenthesis
2. Remove annotations:
 - Those that don't contain html
 - Those that cannot be found in the html (using the same processing they do on the html) 
 - Those annotations that are pointing to links (contain http)
3. Format and save the data into the appropriate SWDE format
---
website = domain

In [1]:
import pandas as pd
import pandavro as pdx
from ast import literal_eval
from pathlib import Path
import os   
import sys
import shutil
from tqdm import tqdm
from markuplmft.fine_tuning.run_swde.prepare_data import get_dom_tree

# Load

In [2]:
dataset = 'develop'

## Full Data

In [3]:
data_path = f"../../../web-annotation-extractor/data/processed/{dataset}/dataset.avro"
df = pdx.read_avro(data_path)
df.annotations = df.annotations.apply(literal_eval)
len(df)

75825

## Positive Data (containing at least one annotation)

In [7]:
# Count PAST CLIENT annotations
df['annotations_len'] = df['annotations'].apply(lambda x: x.get("PAST_CLIENT", [])).apply(len)

df_positives = df[df['annotations_len'] > 0]
df_negatives = df[df['annotations_len'] == 0]

negative_fraction = 0.10
df_negatives = df_negatives[df_negatives['domain'].isin(df_positives['domain'])]
df_negatives_sample = df_negatives.groupby("domain").sample(frac=negative_fraction, random_state=66)

# save_path = data_path.replace('.avro', f'_pos({len(df_positives)}).pkl')
# print(f"Saving file: {save_path}")
# df_positives.to_pickle(save_path)

In [8]:
# # data_path = save_path
# if dataset == 'train':
#     data_path = "../../../web-annotation-extractor/data/processed/train/dataset_pos(?)_neg(?).pkl"
# if dataset == 'develop':
#     data_path = "../../../web-annotation-extractor/data/processed/develop/dataset_pos(1986)_neg(5122).pkl"

# df = pd.read_pickle(data_path)
# print(f"{dataset} - {data_path}")
# len(df)

# Select Tags

In [9]:
annotation_tags = ["PAST_CLIENT", "OFFICE_LOCATION", "CASE_STUDY"][:1]
annotation_tags

['PAST_CLIENT']

# Clean Data

## Remove hiphen from domains

In [10]:
df_positives.domain = df_positives.domain.apply(lambda x: x.replace('-', ''))
df_positives_initial_len = len(df_positives)

/Users/aimoredutra/miniconda3/envs/markuplmft/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


## Assert that domain don't contain parenthesis


In [11]:
assert len(df_positives[df_positives['domain'].apply(lambda x: '(' in x or ')' in x)]) == 0
# Remove parenthesis from domain in case there is (assertion above fails)
# ... df = df[~df['domain'].apply(lambda x: '(' in x or ')' in x)]

# Get text annotations per tag

In [12]:
for tag in annotation_tags:
    print(f'- {tag}')
    df_positives[f'annotations-{tag}'] = df_positives['annotations'].apply(lambda x: x.get(tag))        
    df_positives[f'text-{tag}'] = df_positives.dropna(subset=[f'annotations-{tag}'])[f'annotations-{tag}'].apply(lambda x: [y['text'] for y in x])        

- PAST_CLIENT


/Users/aimoredutra/miniconda3/envs/markuplmft/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/aimoredutra/miniconda3/envs/markuplmft/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


# Remove samples that don't have html

In [13]:
pages_without_html = df_positives[df_positives['html'] == 'PLACEHOLDER_HTML'].dropna(subset=['annotations-PAST_CLIENT'])
annotations_without_html = len([y for x in pages_without_html['text-PAST_CLIENT'] for y in x])
print(f"There are {len(pages_without_html)} pages and {annotations_without_html} annotations without html - We are loosing these annotations because we don't have the html")

There are 156 pages and 158 annotations without html - We are loosing these annotations because we don't have the html


In [14]:
df_positives = df_positives[df_positives['html'] != 'PLACEHOLDER_HTML']

# Remove annotations that cannot be found in the xpaths of the html

In [15]:
initial_amount_of_label = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Initial amount of labels: {initial_amount_of_label}")

all_new_annotations = []

for i, row in tqdm(df_positives.iterrows()):
    if not row.isnull()[f'text-{tag}']:
        clean_dom_tree = get_dom_tree(row['html'], 'website')
        
        annotations_that_can_be_found = []
        annotations_that_cannot_be_found = []
        for text_annotation in row[f'text-{tag}']:
            for node in clean_dom_tree.iter():
                if node.text:
                    if text_annotation in node.text:
                        annotations_that_can_be_found.append(text_annotation)
                        break
                if node.tail:
                    if text_annotation in node.tail:
                        annotations_that_can_be_found.append(text_annotation)
                        break
                # for html_tag, xpath_content in node.items():
                #     if text_annotation in xpath_content:
                #         annotations_that_can_be_found.append(text_annotation)
                #         break
            annotations_that_cannot_be_found.append(text_annotation)
            
        if len(annotations_that_cannot_be_found) > 0:
            print(f"Cannot be found in {i} \t: {annotations_that_cannot_be_found}")
            print()

        all_new_annotations.append(annotations_that_can_be_found)
    else:
        all_new_annotations.append(None)

df_positives[f'text-{tag}'] = all_new_annotations

final_amount_of_label = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])

In [15]:
print(f"Final amount of labels: {final_amount_of_label}")
print(f"Number of labels lost because they couldn't be found in the page: {initial_amount_of_label - final_amount_of_label}")

Initial amount of labels: 3367


2it [00:00, 12.37it/s]

Cannot be found in 1 	: ['CBCF', 'Dallas Morning News', 'BET', 'GoDo Discovery Co.', 'GDC Marketing & Ideation', 'Texas Veterans Land Board', 'Slice', 'Dallas Citizens Council', 'Belmont Icehouse', 'RyanMail', 'TM Advertising', 'Home Matters', 'McCormick Law Firm', 'Good With Everything', 'GCG', 'The Richards Group', 'Schaefer Advertising', 'OCGPR+', 'The Pillow Bar', 'Manpower', '7-Eleven Gameboard', 'Emily', 'TCC', 'Dance', 'TRWD', 'Atmos Energy', 'Embassy City', 'Parkland Foundation', 'Fleet', 'Balfour Beatty', 'DUDA', 'Wingstop', 'St. Philip’s', 'DCVB', 'Sports Dads', 'Source Support', 'Daughter', 'Vizient', 'Mary Kay', 'Freeman AV', 'Boom Boom Satellites', 'Rodeo', 'Mega Kids']

Cannot be found in 2 	: ['THE BROOKLYN CIRCUS']

Cannot be found in 4 	: ['auxmoney']

Cannot be found in 6 	: ['Hampshire Trust Bank']



5it [00:00, 17.63it/s]

Cannot be found in 7 	: ['tsB']

Cannot be found in 9 	: ['Ball Corporation', 'Level 3', 'United Launch Alliance', 'APC Automotive Technologies']

Cannot be found in 10 	: ['ConAgra Foods', 'Sony', 'HP', 'VF Corp', 'Waste Management', 'Direct Energy']



...

Cannot be found in 9287 	: ['Hamburg Commercial Bank']

Cannot be found in 9288 	: ['Obton']

Cannot be found in 9289 	: ['Jobandtalent']

Cannot be found in 9290 	: ['Terra Firma']

Cannot be found in 9291 	: ['TK Elevator']



1829it [01:54, 21.59it/s]

Cannot be found in 9292 	: ['Statkraft']



1830it [01:55, 15.91it/s]

Cannot be found in 9332 	: ['vodafone', 'cdc', 'hertz', 'circle', 'mcbride', 'jefferies', 'primary', 'tkmax', 'moneycorp', 'erm', 'groupm', 'imas', 'impilo', 'sage', 'rentify', 'achica', 'investec', 'aa', 'google', 'inflexion', 'centrica', 'novartis', 'grosvenor', 'ingenious', 'electronics', 'eqt', 'netjets', 'corestate', 'chelsfield', 'care', 'stephens', 'CDG', 'burgess', 'cinven', 'trispan', 'hospedia', 'cobham', 'kingfisher', 'cyrus', 'baird', 'heron', 'gan', 'laurel', 'hermes', 'ares', 'triton', 'schindler', 'viacom', 'gaucho', 'specsavers', 'inchcape', 'kelkoo', 'santander', 'genus', 'bluegem', 'intertek', 'grey', 'berkeley', 'clarion', 'omers', 'gmt', 'sky', 'phoenix', 'efront', 'sodexo', 'rileys', 'montagu', 'which', 'bowmark', 'coach', 'lgc', 'Senior', 'first', 'clearwater', 'paypal', 'pearson', 'permira', 'mec', 'crosstree', 'williams', 'ascential', 'Investment Partner', 'cerberus', 'alchemy', 'cbre', 'rutland', 'kaz', 'diageo', 'econet', 'itn', 'merlin', 'tui', 'ox', 'game', 

In [16]:
current_annotations = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Annotations: {current_annotations} | Pages: {len(df_positives)}")

Annotations: 2533 | Pages: 1830


In [19]:
print(f"With text and tail the page coverage is: {100*len(df_positives)/df_positives_initial_len:.2f} %")

With text and tail the page coverage is: 92.15 %


In [20]:
# print(f"With text, tail and xpath_content the annotation coverage is: {100*11500/12085:.2f} %") # outdated

## Debug (find why annotation could not be found in html):

In [21]:
# row = df.iloc[3143]
# print(row)
# clean_dom_tree = get_dom_tree(row['html'], 'website')
# print()
# for node in clean_dom_tree.iter():
#     for html_tag, xpath_content in node.items():
#         print('text:', node.text)
#         print('tail:', node.tail)
#         print('xpath:', xpath_content)        

In [22]:
# html = row['html']

# Remove image link annotations

In [23]:
current_annotations = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Annotations: {current_annotations} | Pages: {len(df_positives)}")

Annotations: 2533 | Pages: 1830


In [24]:
df_positives[f'text-{tag}'] = df_positives[f'text-{tag}'].dropna().apply(lambda annotations: [annotation  for annotation in annotations if 'http' not in annotation])

In [25]:
current_annotations = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Annotations: {current_annotations} | Pages: {len(df_positives)}")

Annotations: 2533 | Pages: 1830


# Remove samples without annotation

In [26]:
df_positives = df_positives[df_positives[f'text-{tag}'].fillna('').apply(list).apply(len) > 0]

In [27]:
current_annotations = len([y for x in df_positives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Annotations: {current_annotations} | Pages: {len(df_positives)}")

Annotations: 2533 | Pages: 1735


# Add negatives back

In [28]:
df_positives_negatives = df_positives.append(df_negatives_sample)

# Save intermediate Data

In [29]:
data_path

'../../../web-annotation-extractor/data/processed/develop/dataset.avro'

In [32]:
save_path = data_path.replace('.avro', f'_pos({len(df_positives)})_neg({len(df_negatives_sample)})_intermediate.pkl')
print(f"Saving file: {save_path}")
df_positives_negatives.to_pickle(save_path)

save_path = data_path.replace(".avro", f"_pos({len(df_positives)})_intermediate.pkl")
print(f"Saving file: {save_path}")
df_positives.to_pickle(save_path)

Saving file: ../../../web-annotation-extractor/data/processed/develop/dataset_pos(1735)_neg(5122)_intermediate.pkl
Saving file: ../../../web-annotation-extractor/data/processed/develop/dataset_pos(1735)_intermediate.pkl


# Format and Save data

In [ ]:
raw_data_folder = Path.cwd().parents[2] / f'swde/my_data/{dataset}/my_CF_sourceCode'

if os.path.exists(raw_data_folder):
    print(f'Are you sure you want to remove this folder? (y/n) \n{raw_data_folder}')
    answer = input()    
    if answer == 'y':
        try:
            shutil.rmtree(raw_data_folder)
            print(f"REMOVED: {raw_data_folder}")
        except OSError as e:
            print ("Error: %s - %s." % (e.filename, e.strerror))
# else:
#     print(f"File '{groundtruth_data_path}' not found in the directory")
    
domains = list(df.domain.value_counts().index)

groundtruth_data_path = raw_data_folder / 'groundtruth'
groundtruth_data_path.mkdir(parents=True, exist_ok=True)

for e, domain in enumerate(domains):        
    df_domain = df_positives_negatives[df_positives_negatives.domain == domain]
    
    print(f"{e:>3}: {len(df_domain):>5} page(s) - {domain:>25}")
    
    domain_annotations = {}    
    
    page_count = 0
    domain_len = len(df_domain)
    
    for enum, df_page in df_domain.iterrows():
        # Save html
        html = df_page['html']
        raw_data_path = raw_data_folder / 'WAE' / f"{domain}({domain_len})"
        raw_data_path.mkdir(parents=True, exist_ok=True)
        raw_data_path = (raw_data_path / str(page_count).zfill(4)).with_suffix('.htm')
        print(raw_data_path)
        
        Html_file = open(raw_data_path, "w")
        Html_file.write(html)
        Html_file.close()
        
        page_count += 1
                
        # Get groundtruth for page for each tag
        for tag in annotation_tags:            
            domain_annotations[tag] = domain_annotations.get(tag, [])
            if not df_page.isnull()[f'text-{tag}']:
                annotations = df_page[f'text-{tag}']                
                # Remove image links from text annotation
                annotate = [annotation.strip() if (annotation and 'http' not in annotation.strip()) else '' for annotation in annotations]
            else:
                annotate = []
            # print(f'annotate: \n{annotate} - {len(annotate)}')            
            domain_annotations[tag].append(annotate)            
        print()
        # if raw_data_path.name == '0042.htm':
        #     break
        
    # Save groundtruth    
    for tag, page_annotations in domain_annotations.items():
        groundtruth_data_tag_path = groundtruth_data_path / f"{domain}-{tag}.txt"
        print(groundtruth_data_tag_path)

        page_annotations_df = pd.DataFrame(page_annotations)
        
        # Count number of annotations
        page_annotations_df['number of values'] = page_annotations_df.T.count()        
        
        # Invert columns order 
        cols = page_annotations_df.columns.tolist()
        page_annotations_df = page_annotations_df[cols[::-1]] 
        
        # Get page index
        page_annotations_df.reset_index(inplace=True)
        page_annotations_df['index'] = page_annotations_df['index'].apply(lambda x: str(x).zfill(4))
        
        # Add one extra row on the top
        page_annotations_df.loc[-1] = page_annotations_df.count()  # adding a row
        page_annotations_df.index = page_annotations_df.index + 1  # shifting index
        page_annotations_df = page_annotations_df.sort_index()
        
        page_annotations_df.to_csv(groundtruth_data_tag_path, sep="\t", index=False)

# Final Stats

## Number of Domains

In [34]:
df_positives_negatives.domain.value_counts()

greatplacetowork.com    459
esriuk.com              286
outsystems.com          227
informatica.com         223
diligent.com            164
                       ... 
ntouchstrategies.com      2
captionaccess.com         2
berkleygroup.com          1
canelamedia.com           1
metabroadcast.com         1
Name: domain, Length: 105, dtype: int64

## Number of Pages

In [33]:
current_annotations = len([y for x in df_positives_negatives.dropna(subset=[f'text-{tag}'])[f'text-{tag}'].values for y in x])
print(f"Annotations: {current_annotations} | Pages: {len(df_positives_negatives)}")

Annotations: 2533 | Pages: 6857


---